In [32]:
import pandas as pd
# 读取C:\MyProject\MIMIC-IPE\加了中介变量的数据集\备用数据集.csv
df = pd.read_csv(r'C:\MyProject\MIMIC-IPE\加了中介变量的数据集\备用数据集.csv')
# 查看df的所有列名，存入一个dataframe，然后计算每列的空值计数以及空值占比，也存入这个dataframe
# 获取df的所有列名，存入一个dataframe
columns_df = pd.DataFrame(df.columns, columns=['column_name'])
# 计算每列的空值计数
null_counts = df.isnull().sum()
# 计算每列的空值占比
null_ratios = df.isnull().mean() * 100
# 将空值计数和空值占比添加到columns_df中
columns_df['null_counts'] = null_counts.values
columns_df['null_ratios'] = null_ratios.values

'''
以下几列的空值使用该变量中中位数填充
Time of death (days)
NRPE event time span
Death due to PE event time span
'''
# List of columns to fill null values with the median
columns_to_fill = ['Time of death (days)', 'NRPE event time span', 'Death due to PE event time span']

# Fill null values in the specified columns with the max value
for column in columns_to_fill:
    max_value = df[column].max()
    df[column] = df[column].fillna(max_value)
'''
以下几列的空值使用0填充
Direct thrombin inhibitor
Factor Xa inhibitor
Unfractionated heparin
Vitamin K antagonist
icu_count_time
icu_total_los_day
icu_total_hour
Total ICU stay (hours)
icu_count_num
'''
# List of columns to fill null values with 0
columns_to_fill = ['Direct thrombin inhibitor', 'Factor Xa inhibitor', 'Unfractionated heparin', 'Vitamin K antagonist', 'icu_count_time', 'icu_total_los_day', 'icu_total_hour', 'Total ICU stay (hours)', 'icu_count_num']
# Fill null values in the specified columns with 0
for column in columns_to_fill:
    df[column] = df[column].fillna(0)
# 查看df的所有列名，存入一个dataframe，然后计算每列的空值计数以及空值占比，也存入这个dataframe
# 获取df的所有列名，存入一个dataframe
columns_df = pd.DataFrame(df.columns, columns=['column_name'])
# 计算每列的空值计数
null_counts = df.isnull().sum()
# 计算每列的空值占比
null_ratios = df.isnull().mean() * 100
# 将空值计数和空值占比添加到columns_df中
columns_df['null_counts'] = null_counts.values
columns_df['null_ratios'] = null_ratios.values
'''
删除数据集中的以下列：
death_time_days
avg_los
death_days
race_AMERICAN INDIAN/ALASKA NATIVE
race_ASIAN
race_ASIAN - CHINESE
race_BLACK/AFRICAN
race_BLACK/AFRICAN AMERICAN
race_BLACK/CAPE VERDEAN
race_BLACK/CARIBBEAN ISLAND
race_HISPANIC OR LATINO
race_HISPANIC/LATINO - COLUMBIAN
race_HISPANIC/LATINO - DOMINICAN
race_HISPANIC/LATINO - GUATEMALAN
race_HISPANIC/LATINO - PUERTO RICAN
race_MULTIPLE RACE/ETHNICITY
race_OTHER
race_PATIENT DECLINED TO ANSWER
race_PORTUGUESE
race_SOUTH AMERICAN
race_UNABLE TO OBTAIN
race_UNKNOWN
race_WHITE
race_WHITE - EASTERN EUROPEAN
race_WHITE - OTHER EUROPEAN
race_WHITE - RUSSIAN
icu_count_time
icu_total_hour
re_inhosp_rehosp
seq_num
icu_count_num
'''
# List of columns to be deleted
columns_to_delete = [

    "avg_los",
    
    "race_AMERICAN INDIAN/ALASKA NATIVE",
    "race_ASIAN",
    "race_ASIAN - CHINESE",
    "race_BLACK/AFRICAN",
    "race_BLACK/AFRICAN AMERICAN",
    "race_BLACK/CAPE VERDEAN",
    "race_BLACK/CARIBBEAN ISLAND",
    "race_HISPANIC OR LATINO",
    "race_HISPANIC/LATINO - COLUMBIAN",
    "race_HISPANIC/LATINO - DOMINICAN",
    "race_HISPANIC/LATINO - GUATEMALAN",
    "race_HISPANIC/LATINO - PUERTO RICAN",
    "race_MULTIPLE RACE/ETHNICITY",
    "race_OTHER",
    "race_PATIENT DECLINED TO ANSWER",
    "race_PORTUGUESE",
    "race_SOUTH AMERICAN",
    "race_UNABLE TO OBTAIN",
    "race_UNKNOWN",
    "race_WHITE",
    "race_WHITE - EASTERN EUROPEAN",
    "race_WHITE - OTHER EUROPEAN",
    "race_WHITE - RUSSIAN",
    "icu_count_time",
    "icu_total_hour",
    "re_inhosp_rehosp",
    "seq_num",
    "icu_count_num"
]

# Delete the columns from the dataframe
df = df.drop(columns=columns_to_delete)
# 将hospital_expire_flag列更名为'In-hospital deaths'
df = df.rename(columns={'hospital_expire_flag': 'In-hospital deaths'})
# 检查df中的列名，如果列名的首字母不是大写字母，则将其更改为大写字母
# Check the column names in df, if the first letter of the column name is not an uppercase letter, change it to an uppercase letter
df.columns = [column.capitalize() if column[0].islower() else column for column in df.columns]

# df中生成3个新的列，用于生存分析。分别是‘Time to death within 7 days’，‘Time to death within 14 days’，‘Time to death within 31 days’，这三个列的值来自‘Time of death (days)’列，在Time to death within 7 days中，将‘Time of death (days)’列中的值小于等于7的行的直接对应过来，大于7的行的值为7，其他两列同理
# Create 'Time to death within 7 days' column
df['Time to death within 7 days'] = df['Time of death (days)'].apply(lambda x: x if x <= 7 else 7)

# Create 'Time to death within 14 days' column
df['Time to death within 14 days'] = df['Time of death (days)'].apply(lambda x: x if x <= 14 else 14)

# Create 'Time to death within 31 days' column
df['Time to death within 31 days'] = df['Time of death (days)'].apply(lambda x: x if x <= 31 else 31)
# 检查df中的列名，如果列名中包含'_'、'-'、'-'、'_'以及空格,将这些符合替换为一个'_'
# Check the column names in df, if the column name contains '_', '-', ' ', and '-', replace these with a '_'

df.columns = [column.replace('_', ' ').replace('-', ' ').replace(' ', '_') for column in df.columns]
# 删除列名中的中文和英文格式括号
df.columns = [column.replace('（', '').replace('）', '') for column in df.columns]
# 删除列名中的英文格式括号
df.columns = [column.replace('(', '').replace(')', '') for column in df.columns]
# 将数据保存到原路径，覆盖原数据
df.to_csv(r'C:\MyProject\MIMIC-IPE\加了中介变量的数据集\用于统计的完整数据集.csv', index=False)
import pandas as pd
# 读取C:\MyProject\MIMIC-IPE\加了中介变量的数据集\用于统计的完整数据集.csv
df = pd.read_csv(r'C:\MyProject\MIMIC-IPE\加了中介变量的数据集\用于统计的完整数据集.csv')
# 读取C:\MyProject\MIMIC-IPE\data\住院次序.csv
stay_df = pd.read_csv(r'C:\MyProject\MIMIC-IPE\data\住院次序.csv')
# 将stay_df中列名如果不是大写都改成大写
stay_df.columns = [column.capitalize() if column[0].islower() else column for column in stay_df.columns]
# 将stay_df中的seq_hosp列更名为Hospitalization sequence
stay_df = stay_df.rename(columns={'Seq_hosp': 'Hospitalization sequence'})
# 将stay_df按照subject_id和hadm_id与df合并
df = pd.merge(df, stay_df, on=['Subject_id', 'Hadm_id'], how='left')
# 生存分析：使用df中的dod列减去Admittime列，得到的时间差以月为单位，我现在要增加几个列，分别是1年内死亡的患者的死亡时间和死亡事件，注意，这里的1年是指12个月，注意存活超过1年的患者的生存时间采取右截尾处理，即将其生存时间设置为12个月
# Ensure 'dod' and 'Admittime' are in datetime format
df['Dod'] = pd.to_datetime(df['Dod'])
df['Admittime'] = pd.to_datetime(df['Admittime'])

# Calculate the time difference in days and convert to months
df['time_diff'] = (df['Dod'] - df['Admittime']).dt.days / 30.44

# Create new columns for death time and event within 1 year
df['death_time_within_1_year'] = df['time_diff'].apply(lambda x: x if x <= 12 else 12)
df['death_event_within_1_year'] = df['time_diff'].apply(lambda x: 1 if x <= 12 else 0)
# Create new columns for death time and event within 3 years
df['death_time_within_3_years'] = df['time_diff'].apply(lambda x: x if x <= 36 else 36)
df['death_event_within_3_years'] = df['time_diff'].apply(lambda x: 1 if x <= 36 else 0)

# Create new columns for death time and event within 5 years
df['death_time_within_5_years'] = df['time_diff'].apply(lambda x: x if x <= 60 else 60)
df['death_event_within_5_years'] = df['time_diff'].apply(lambda x: 1 if x <= 60 else 0)
# 生存分析：使用df中的dod列减去Admittime列，得到的时间差以天为单位
# Ensure 'dod' and 'Admittime' are in datetime format


# Calculate the time difference in days
df['time_diff_day'] = (df['Dod'] - df['Admittime']).dt.days

# Create new columns for death time and event within 7 days
df['death_time_within_7_days'] = df['time_diff_day'].apply(lambda x: x if x <= 7 else 7)
df['death_event_within_7_days'] = df['time_diff_day'].apply(lambda x: 1 if x <= 7 else 0)

# Create new columns for death time and event within 14 days
df['death_time_within_14_days'] = df['time_diff_day'].apply(lambda x: x if x <= 14 else 14)
df['death_event_within_14_days'] = df['time_diff_day'].apply(lambda x: 1 if x <= 14 else 0)

# Create new columns for death time and event within 31 days
df['death_time_within_31_days'] = df['time_diff_day'].apply(lambda x: x if x <= 31 else 31)
df['death_event_within_31_days'] = df['time_diff_day'].apply(lambda x: 1 if x <= 31 else 0)

# Create new columns for death time and event within 60 days
df['death_time_within_60_days'] = df['time_diff_day'].apply(lambda x: x if x <= 60 else 60)
df['death_event_within_60_days'] = df['time_diff_day'].apply(lambda x: 1 if x <= 60 else 0)

# Create new columns for death time and event within 3 months
df['death_time_within_3_months'] = df['time_diff_day'].apply(lambda x: x if x <= 90 else 90)
df['death_event_within_3_months'] = df['time_diff_day'].apply(lambda x: 1 if x <= 90 else 0)

# Create new columns for death time and event within 6 months
df['death_time_within_6_months'] = df['time_diff_day'].apply(lambda x: x if x <= 180 else 180)
df['death_event_within_6_months'] = df['time_diff_day'].apply(lambda x: 1 if x <= 180 else 0)
# 获取df的所有列名，存入一个dataframe
columns_df = pd.DataFrame(df.columns, columns=['column_name'])
'''
df中这几个列目前是以天为单位的，改成以月为单位，并且相应的修改列名
Time_of_death_days
NRPE_event_time_span
UNRPE_event_time_span

'''
# List of columns to convert from days to months
columns_to_convert = [
    'Time_of_death_days',
    'NRPE_event_time_span',
    'UNRPE_event_time_span',
    
]

# Convert the units from days to months and rename the columns
for column in columns_to_convert:
    df[column] = df[column] / 30.44
    df.rename(columns={column: column.replace('_days', '_months')}, inplace=True)

'''
df中这几个列目前是以小时为单位的，改成以月为单位，并且相应的修改列名
Death_due_to_PE_event_time_span
'''
# Convert the 'Death_due_to_PE_event_time_span' from hours to months
df['Death_due_to_PE_event_time_span'] = df['Death_due_to_PE_event_time_span'] / (24 * 30.44)

# Rename the column
df.rename(columns={'Death_due_to_PE_event_time_span': 'Death_due_to_PE_event_time_span_months'}, inplace=True)

# 查看df中所有的列名，使用一个dataframe存储，然后计算每列的空值计数以及空值占比，也存入这个dataframe
# 获取df的所有列名，存入一个dataframe
columns_df = pd.DataFrame(df.columns, columns=['column_name'])
# 计算每列的空值计数
null_counts = df.isnull().sum()
# 计算每列的空值占比
null_ratios = df.isnull().mean() * 100
# 将空值计数和空值占比添加到columns_df中
columns_df['null_counts'] = null_counts.values
columns_df['null_ratios'] = null_ratios.values

'''调整df的以下列：
Death_event——>D.E.
Time_of_death_months——>T.O.D. (mth)
Death_time_days——>T.O.D. (d)
death_event_within_7_days——>D7D
death_time_within_7_days——>T.O.D. (7d)
death_event_within_14_days——>D14D
death_time_within_14_days——>T.O.D. (14d)
death_event_within_31_days——>D31D
death_time_within_31_days——>T.O.D. (31d)
death_time_within_60_days——>T.O.D. (60d)
death_event_within_60_days——>D60D
death_time_within_3_months——>T.O.D. (90d)
death_event_within_3_months——>D90D
death_time_within_6_months——>T.O.D. (180d)
death_event_within_6_months——>D180D
death_event_within_1_year——>D12M
death_time_within_1_year——>T.O.D. (12mth)
death_event_within_3_years——>D36M
death_time_within_3_years——>T.O.D. (36mth)
death_event_within_5_years——>D60M
death_time_within_5_years——>T.O.D. (60mth)
Count_of_ICU_stays——>Cnt. ICU
Total_ICU_stay_hours——>Tot. ICU (hrs)
First_hospitalization——>F. Hosp
Length_of_hospital_stay_hours——>LOHS (hrs)
NRPE_event_time_span——>NRPE-ETS(mth)
UNRPE_event_time_span——>UNRPE-ETS(mth)
Death_due_to_first_admission_for_PE——>Death-1st adm.-PE
Death_due_to_PE ——>Death-PE
Death_due_to_PE_event_time_span_months——>Death-PE ETS(mth)
'''
# Define a dictionary for the column name changes
column_name_changes = {
    'Group_Label': 'Group',
    'Death_event': 'D.E.',
    'Time_of_death_months': 'T.O.D_mth',
    'Death_time_days': 'T.O.D_d',
    'death_event_within_7_days': 'D7D',
    'death_time_within_7_days': 'T.O.D_7d',
    'death_event_within_14_days': 'D14D',
    'death_time_within_14_days': 'T.O.D_14d',
    'death_event_within_31_days': 'D31D',
    'death_time_within_31_days': 'T.O.D_31d',
    'death_time_within_60_days': 'T.O.D_60d',
    'death_event_within_60_days': 'D60D',
    'death_time_within_3_months': 'T.O.D_90d',
    'death_event_within_3_months': 'D90D',
    'death_time_within_6_months': 'T.O.D_180d',
    'death_event_within_6_months': 'D180D',
    'death_event_within_1_year': 'D12M',
    'death_time_within_1_year': 'T.O.D_12mth',
    'death_event_within_3_years': 'D36M',
    'death_time_within_3_years': 'T.O.D_36mth',
    'death_event_within_5_years': 'D60M',
    'death_time_within_5_years': 'T.O.D_60mth',
    'Count_of_ICU_stays': 'Cnt.ICU',
    'Total_ICU_stay_hours': 'Tot.ICU_hrs',
    'First_hospitalization': 'F.Hosp',
    'Length_of_hospital_stay_hours': 'LOHS_hrs',
    'NRPE_event_time_span': 'NRPE_ETS_mth',
    'UNRPE_event_time_span': 'UNRPE_ETS_mth',
    'Death_due_to_first_admission_for_PE': 'Death.1st.adm.PE',
    'Death_due_to_PE': 'Death.PE',
    'Death_due_to_PE_event_time_span_months': 'Death.PE.ETS_mth'
}

# Rename the columns
df.rename(columns=column_name_changes, inplace=True)
# 查看df中所有的列名，使用一个dataframe存储，然后计算每列的空值计数以及空值占比，也存入这个dataframe
# 获取df的所有列名，存入一个dataframe
columns_df = pd.DataFrame(df.columns, columns=['column_name'])
# 计算每列的空值计数
null_counts = df.isnull().sum()
# 计算每列的空值占比
null_ratios = df.isnull().mean() * 100
# 将空值计数和空值占比添加到columns_df中
columns_df['null_counts'] = null_counts.values
columns_df['null_ratios'] = null_ratios.values
'''
删除df中以下几个列：
Death_days
Deaths_within_7_days
Deaths_within_14_days
Deaths_within_31_days
Hospstay_seq
Icu_total_los_day
Seq_hosp
Time_to_death_within_7_days
Time_to_death_within_14_days
Time_to_death_within_31_days
time_diff
'''
# List of columns to be deleted
columns_to_delete = [
    "Death_days",
    "Deaths_within_7_days",
    "Deaths_within_14_days",
    "Deaths_within_31_days",
    "Hospstay_seq",
    "Icu_total_los_day",
    "Seq_hosp",
    "Time_to_death_within_7_days",
    "Time_to_death_within_14_days",
    "Time_to_death_within_31_days",
    "time_diff"
]

# Delete the columns from the dataframe
df = df.drop(columns=columns_to_delete)
'''
将以下几列也列名也进行调整：
Direct_thrombin_inhibitor：DTI
Factor_Xa_inhibitor：FXaI
Unfractionated_heparin：UFH
Vitamin_K_antagonist：VKA
'''
# Define a dictionary for the column name changes
column_name_changes = {
    'Direct_thrombin_inhibitor': 'DTI',
    'Factor_Xa_inhibitor': 'FXaI',
    'Unfractionated_heparin': 'UFH',
    'Vitamin_K_antagonist': 'VKA'
}

# Rename the columns
df.rename(columns=column_name_changes, inplace=True)
# 将以上几个列中的值也进行更改，1改为“Administered”，0改为“Unadministered”
# List of columns to change the values
columns_to_change = ['DTI', 'FXaI', 'UFH', 'VKA']

# Change the values in the specified columns
for column in columns_to_change:
    df[column] = df[column].apply(lambda x: 'Administered' if x == 1 else 'Unadministered')
    
# 做一个适合描述性统计的
# 首先查看df中所有的的列名，以及列名对应的缺失值和百分比
# 获取df的所有列名，存入一个dataframe
columns_df = pd.DataFrame(df.columns, columns=['column_name'])
# 计算每列的空值计数
null_counts = df.isnull().sum()
# 计算每列的空值占比
null_ratios = df.isnull().mean() * 100
# 将空值计数和空值占比添加到columns_df中
columns_df['null_counts'] = null_counts.values
columns_df['null_ratios'] = null_ratios.values


df['Race'] = df['Race'].replace('Unknown or Declined', None)
# 将Gender列中F改为Female，M改为Male
df['Gender'] = df['Gender'].replace({'F': 'Female', 'M': 'Male'})





In [33]:
'''
Cardiovascular_and_metabolic_diseases列：
No Cardiovascular and metabolic diseases
Hypertension
Atrial fibrillation
Heart failure
Diabetes
CAD
Blood_related_diseases列：
No Blood-related disease
Primary Thrombocytopenia
Thrombotic Thrombocytopenic Purpura
Other Coagulation Defects
Tumor_disease列：
No tumor disease
Other_Solid_Tumor
Lung_Tumor
Non_Solid_Tumor
'''
'''
Cardiovascular and Metabolic Diseases (CVD_MD)
No Cardiovascular and metabolic diseases: No CVD_MD
Hypertension: HTN
Atrial fibrillation: AF
Heart failure: HF
Diabetes: DM
CAD (Coronary Artery Disease): CAD
Blood Related Diseases (BRD)
No Blood-related disease: No BRD
Primary Thrombocytopenia: PT
Thrombotic Thrombocytopenic Purpura: TTP
Other Coagulation Defects: OCD
Tumor Disease (TD)
No tumor disease: No TD
Other Solid Tumor: OST
Lung Tumor: LT
Non-Solid Tumor: NST
'''
# Define a dictionary for the column name changes
column_name_changes = {
    'Cardiovascular_and_metabolic_diseases': 'CVD_MD',
    'Blood_related_diseases': 'BRD',
    'Tumor_disease': 'TD'
}

# Rename the columns
df.rename(columns=column_name_changes, inplace=True)

# Define dictionaries for the value changes
cvd_md_value_changes = {
    'No Cardiovascular and metabolic diseases': 'No CVD_MD',
    'Hypertension': 'HTN',
    'Atrial fibrillation': 'AF',
    'Heart failure': 'HF',
    'Diabetes': 'DM',
    'CAD': 'CAD'
}

brd_value_changes = {
    'No Blood-related disease': 'No BRD',
    'Primary Thrombocytopenia': 'PT',
    'Thrombotic Thrombocytopenic Purpura': 'TTP',
    'Other Coagulation Defects': 'OCD'
}

td_value_changes = {
    'No tumor disease': 'No TD',
    'Other Solid Tumor': 'OST',
    'Lung Tumor': 'LT',
    'Non-Solid Tumor': 'NST'
}

# Replace the values in the columns
df['CVD_MD'] = df['CVD_MD'].replace(cvd_md_value_changes)
df['BRD'] = df['BRD'].replace(brd_value_changes)
df['TD'] = df['TD'].replace(td_value_changes)
# 把以上几列one-hot化，并且使用1和0替换True和False
# List of columns to one-hot encode
columns_to_one_hot = ['CVD_MD', 'BRD', 'TD']

# One-hot encode the columns
df = pd.get_dummies(df, columns=columns_to_one_hot)

# Replace True and False with 1 and 0
df = df.replace({True: 'True', False: 'False'})


In [34]:
# F.Hosp列中值更改：1改为“Affirmative”，0改为“Negative”
df['F.Hosp'] = df['F.Hosp'].apply(lambda x: 'Affirmative' if x == 1 else 'Negative')

In [35]:
# 读取C:\MyProject\MIMIC-IPE\data\住院时间与ICU时间.csv
stay_time_df = pd.read_csv(r'C:\MyProject\MIMIC-IPE\data\住院时间与ICU时间.csv')
# 将stay_time_df中的列名首字母改为大写
stay_time_df.columns = [column.capitalize() if column[0].islower() else column for column in stay_time_df.columns]
# 按照Subject_id和Hadm_id合并df和stay_time_df，只保留stay_time_df中的Hosp_time_hour列和Icu_total_hour列
df = pd.merge(df, stay_time_df[['Subject_id', 'Hadm_id', 'Hosp_time_hour', 'Icu_total_hour']], on=['Subject_id', 'Hadm_id'], how='left')

In [36]:

# 将数据保存到原路径，覆盖原数据
df.to_csv(r'C:\MyProject\MIMIC-IPE\加了中介变量的数据集\用于统计的完整数据集.csv', index=False)
# df仅保留stay_id列不为空的行
df = df[df['Stay_id'].notnull()]
# 将数据保存到原路径，覆盖原数据
df.to_csv(r'C:\MyProject\MIMIC-IPE\加了中介变量的数据集\ICU完整数据集.csv', index=False)
# 读取C:\MyProject\MIMIC-IPE\data\更少的数据集.csv
less_df = pd.read_csv(r'C:\MyProject\MIMIC-IPE\data\更少的数据集.csv')
# 使用less_df的subject_id和hadm_id列，筛选df中的数据
df = df[df['Subject_id'].isin(less_df['subject_id']) & df['Hadm_id'].isin(less_df['hadm_id'])]
# 保存df到C:\MyProject\MIMIC-IPE\加了中介变量的数据集\less完整数据集.csv
df.to_csv(r'C:\MyProject\MIMIC-IPE\加了中介变量的数据集\less完整数据集.csv', index=False)
# df仅保留stay_id列不为空的行
df = df[df['Stay_id'].notnull()]
# 将数据保存到原路径，覆盖原数据
df.to_csv(r'C:\MyProject\MIMIC-IPE\加了中介变量的数据集\less_ICU完整数据集.csv', index=False)

In [37]:
# 做一个适合描述性统计的
# 首先查看df中所有的的列名，以及列名对应的缺失值和百分比
